<a href="https://colab.research.google.com/github/jx-dohwan/BERT-FP_MRS/blob/main/fine_tuning/fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# fine-tuning

In [1]:

!pip install torch
!pip install transformers==4.25.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 83.7 MB/s eta 0:00:00


In [2]:

!git clone https://github.com/smilegate-ai/korean_smile_style_dataset
# https://corpus.korean.go.kr/
# https://aihub.or.kr/aihub-data/natural-language/about
     

Cloning into 'korean_smile_style_dataset'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 81 (delta 21), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (81/81), 2.57 MiB | 3.46 MiB/s, done.


In [3]:
import pandas as pd
df = pd.read_csv('/content/korean_smile_style_dataset/smilestyle_dataset.tsv', sep='\t')
df.to_csv('./korean_smile_style_dataset/smile.csv', index=False)

In [4]:
!mv /content/korean_smile_style_dataset/smile.csv /content/drive/MyDrive/인공지능/멀티턴응답선택/korean_smile_style_dataset

## 학습/검증 분할하기|

In [5]:
import csv
f = open('/content/drive/MyDrive/인공지능/멀티턴응답선택/korean_smile_style_dataset/smile.csv', 'r')
rdr = csv.reader(f)

""" 세션 데이터 저장할 것 """
session_dataset = []
session = []
all_utts = []

""" 실제 데이터 저장 방식 """
for i, line in enumerate(rdr):
    if i == 0:
        header = line
    else:
        utt = line[0]
        if utt.strip() != '':
            all_utts.append(utt)
            session.append(utt)
        else:
            """ 세션 데이터 저장 """
            session_dataset.append(session)
            session = []
""" 마지막 세션 저장 """
session_dataset.append(session)
f.close()

In [7]:
!head -20 /content/drive/MyDrive/인공지능/멀티턴응답선택/korean_smile_style_dataset/smile.csv

formal,informal,android,azae,chat,choding,emoticon,enfp,gentle,halbae,halmae,joongding,king,naruto,seonbi,sosim,translator
안녕하세요. 저는 고양이 6마리 키워요.,안녕! 나는 고양이 6마리 키워.,휴먼. 반갑다. 안드로이드는. 고양이. 6마리. 소유중.,아이고 안녕하십니까~ 나는 그냥 고양이 6마리 키우고 있는 사람이여,하잉ㅋㅋ 나 떼걸룩 6마리 키운다!,ㅎㅇ 나 주인님 6마리 모심 ㅋㅋ,안녕!! >< 나는 고양이😺를 ➏ 마리 키우고있어!! 0_0,안녕안녕~! 나 고양이 6마리나 키운다? 완전 대박이징~,"안녕하십니까,, 저는 고양이 6마리 키웁니다.",안녕하신가~... 난 지금 고양이를 6마리 키우고 있다네,하유 시벌것 괭이놈 6마리 키우는데 힘들어 죽겟네,안녕하냐 ㅡㅡ 나 씹냥이 6마리나 키운다 하;,반갑소. 짐은 고양이를 6마리나 키우오.,안녕하냐니깐! 난 고양이를 6마리 키우고있다니깐!,안녕하시오! 소인은 고양이를 6마리 키우고 있소!,안녕… 난 고양이 6마리 키워 ㅠㅠ,반가운. 나는 6마리의 고양이를 소지하고 있다.
고양이를 6마리나요? 키우는거 안 힘드세요?,고양이를 6마리나? 키우는거 안 힘들어?,고양이. 6마리. 양육. 번거로운가.,아니 무슨 고양이를 6마리나? 거 키우는 거 안 힘든가?,엥? 6마리나? 안힘듬?ㅋㅋㅋㅋ,6마리? 에바아니냐 안 힘듦?,고양이를 6마리나?!! w(ﾟДﾟ)w 키우는거 안 힘듬?? (⊙_⊙;),고양이를 6마리나? 완전 대박~ 키우는 거 안 힘들어?!,"고양이를 6마리나 키우십니까? 안 힘드신지,,",고양이를 6마리나? 키우는거 힘들지 않는가?,니기럴 털만 날리는 거 키우기 안 힘들데?,아니 고양이를 6마리나? 안힘드냐?,고양이를 6마리나? 키우는게 수고스럽진 않소?,고양이를 6마리나? 키우는거 힘들지 않냐니깐?,고양이를 6마리나 키우고 있는 것이오? 힘들지 않소?,고양이..6마리나? ㅠ 키우는건 혹시 안힘들어..?,6마리의

In [8]:
print(len(session_dataset))


236


In [9]:
train_session = session_dataset[:-15]
valid_session = session_dataset[-15:]
len(train_session), len(valid_session)

(221, 15)

In [10]:
train_session[0]
# 첫번째 데이터
# context: 안녕하세요. 저는 고양이 6마리 키워요.
# response: 고양이를 6마리나요? 키우는거 안 힘드세요?

# 두번째 데이터
# context: 안녕하세요. 저는 고양이 6마리 키워요. | 고양이를 6마리나요? 키우는거 안 힘드세요?
# response: 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.

# 세번째 데이터
# context: 안녕하세요. 저는 고양이 6마리 키워요. | 고양이를 6마리나요? 키우는거 안 힘드세요? | 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.
# response: 가장 나이가 많은 고양이가 어떻게 돼요?

['안녕하세요. 저는 고양이 6마리 키워요.',
 '고양이를 6마리나요? 키우는거 안 힘드세요?',
 '제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.',
 '가장 나이가 많은 고양이가 어떻게 돼요?',
 '여섯 살입니다. 갈색 고양이에요.',
 '그럼 가장 어린 고양이가 어떻게 돼요?',
 '한 살입니다. 작년에 분양 받았어요.',
 '그럼 고양이들끼리 안 싸우나요?',
 '저희 일곱은 다같이 한 가족입니다. 싸우는 일은 없어요.']

## positive / negative pairs 구성

In [12]:
from collections import defaultdict
train_json = defaultdict(dict)
count = 0
for session in train_session:
    context = [session[0]]
    for turn in range(1, len(session)):
        utt = session[turn] 
        train_json[count]['context'] = context[:]
        train_json[count]['response'] = utt
        train_json[count]['label'] = 1
        context.append(utt)
        count += 1
    break

In [13]:
train_json

defaultdict(dict,
            {0: {'context': ['안녕하세요. 저는 고양이 6마리 키워요.'],
              'response': '고양이를 6마리나요? 키우는거 안 힘드세요?',
              'label': 1},
             1: {'context': ['안녕하세요. 저는 고양이 6마리 키워요.',
               '고양이를 6마리나요? 키우는거 안 힘드세요?'],
              'response': '제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.',
              'label': 1},
             2: {'context': ['안녕하세요. 저는 고양이 6마리 키워요.',
               '고양이를 6마리나요? 키우는거 안 힘드세요?',
               '제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.'],
              'response': '가장 나이가 많은 고양이가 어떻게 돼요?',
              'label': 1},
             3: {'context': ['안녕하세요. 저는 고양이 6마리 키워요.',
               '고양이를 6마리나요? 키우는거 안 힘드세요?',
               '제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.',
               '가장 나이가 많은 고양이가 어떻게 돼요?'],
              'response': '여섯 살입니다. 갈색 고양이에요.',
              'label': 1},
             4: {'context': ['안녕하세요. 저는 고양이 6마리 키워요.',
               '고양이를 6마리나요? 키우는거 안 힘드세요?',
               '제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.',
               '가장 나이가 많은 고양이가 어떻

In [14]:
import random
use_turns = 5 # 꼭 5일 필요는 없음
neg_nums = 4 # 꼭 4일 필요는 없음

from collections import defaultdict
train_json = defaultdict(dict)
count = 0
for session in train_session:
    context = [session[0]]
    for turn in range(1, len(session)):
        utt = session[turn]
        train_json[count]['context'] = context[:][-use_turns:]
        train_json[count]['positive_response'] = utt
        context.append(utt)

        negative_candidates = random.sample(all_utts, neg_nums)
        train_json[count]['negative_responses'] = negative_candidates
        count += 1
    break

In [15]:
train_json

defaultdict(dict,
            {0: {'context': ['안녕하세요. 저는 고양이 6마리 키워요.'],
              'positive_response': '고양이를 6마리나요? 키우는거 안 힘드세요?',
              'negative_responses': ['아, 순간 진짜 모르시는 줄 알고 깜짝 놀랐어요.',
               '군청색을 입으면 악당들의 눈에 안 띄는 만큼 시민들의 눈에 안 띌 가능성도 높겠네요.',
               '그럼 고양이들끼리 안 싸우나요?',
               '언제까지 안 들킬지는 모르겠지만, 재미있는 상상이네요.']},
             1: {'context': ['안녕하세요. 저는 고양이 6마리 키워요.',
               '고양이를 6마리나요? 키우는거 안 힘드세요?'],
              'positive_response': '제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.',
              'negative_responses': ['하지만 1년 전 부터 티켓이 매진되어서, 암표를 구하셔야 될 거에요.',
               '농구를 잘하고 싶으면 여러가지를 잘해야죠.',
               '그러면 저도 오늘 치즈 피자를 먹어보고 싶어지네요.',
               '그렇군요, 전 비틀즈의 헤이 주드를 제일 좋아해요.']},
             2: {'context': ['안녕하세요. 저는 고양이 6마리 키워요.',
               '고양이를 6마리나요? 키우는거 안 힘드세요?',
               '제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.'],
              'positive_response': '가장 나이가 많은 고양이가 어떻게 돼요?',
              'negative_responses': ['그리고 대학교에서는 말 그래도

In [17]:
import random
neg_nums = 4

from collections import defaultdict
train_json = defaultdict(dict)
count = 0
for session in train_session:
    context = [session[0]]
    for turn in range(1, len(session)):
        utt = session[turn]
        train_json[count]['context'] = context[:][-use_turns:]
        train_json[count]['positive_response'] = utt        
        context.append(utt)        
        
        negative_candidates = random.sample(all_utts, neg_nums)
        train_json[count]['negative_responses'] = negative_candidates
        count += 1
    
import json
with open('/content/drive/MyDrive/인공지능/멀티턴응답선택/korean_smile_style_dataset/train.json', 'w') as outfile:
    json.dump(train_json, outfile)

In [18]:
import random
neg_nums = 4

from collections import defaultdict
dev_json = defaultdict(dict)
count = 0
for session in valid_session:
    context = [session[0]]
    for turn in range(1, len(session)):
        utt = session[turn]
        dev_json[count]['context'] = context[:][-use_turns:]
        dev_json[count]['positive_response'] = utt        
        context.append(utt)        
        
        negative_candidates = random.sample(all_utts, neg_nums)
        dev_json[count]['negative_responses'] = negative_candidates
        count += 1
    
import json
with open('/content/drive/MyDrive/인공지능/멀티턴응답선택/korean_smile_style_dataset/dev.json', 'w') as outfile:
    json.dump(dev_json, outfile)